<a href="https://colab.research.google.com/github/dunkelweizen/DS-Unit-2-Kaggle-Challenge/blob/master/Copy_of_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [0]:
#create function to wrangle all datasets the same way
def data_wrangle(dataframe):
    dataframe['gps_height'] = dataframe['gps_height'].astype(float)
    dataframe['longitude'] = dataframe['longitude'].astype(float)
    dataframe['longitude'] = dataframe['longitude'].replace(0, dataframe['longitude'].median())
    dataframe['construction_year'] = dataframe['construction_year'].replace(np.NaN, dataframe['construction_year'].median())
    dataframe['construction_year'] = dataframe['construction_year'].replace(np.NaN, 2010)
    dataframe['wp_age'] = 2019 - dataframe['construction_year']
    dataframe['date_recorded'] = pd.to_datetime(dataframe['date_recorded'])
    dataframe['day_recorded'] = dataframe['date_recorded'].dt.dayofweek
    dataframe['year_recorded'] = dataframe['date_recorded'].dt.year
    dataframe['date_recorded'] = dataframe['date_recorded'].astype(str)
    return dataframe

In [0]:
train['functional'] = (train['status_group']=='functional').astype(int)
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'])

In [7]:
print(train['status_group'].value_counts(normalize=True))
baseline_guess = 'functional'

functional                 0.543077
non functional             0.384238
functional needs repair    0.072685
Name: status_group, dtype: float64


In [0]:
#%% run datasets through function
train = data_wrangle(train)
val = data_wrangle(val)
test = data_wrangle(test)

In [0]:
features = ['wp_age', 'population', 'water_quality', 'day_recorded', 'year_recorded',
                        'extraction_type', 'quantity', 'latitude', 'longitude',
                        'source', 'date_recorded', 'amount_tsh', 'gps_height']
X_train = train[features]
X_val = val[features]
X_test = test[features]

In [60]:
target = 'status_group'

y_train = train[target]
y_val = val[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='most_frequent'),
    RandomForestClassifier(max_features = 7, min_samples_leaf= 1, max_depth=22,
                           min_samples_split= 9, n_estimators= 217, n_jobs=-1)
)
pipeline.fit(X_train, y_train)
print('Training Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

Training Accuracy 0.9227272727272727
Validation Accuracy 0.8090909090909091


In [0]:
from scipy.stats import randint, uniform

In [0]:
encoder = pipeline.named_steps['ordinalencoder']
X_train_encoded = encoder.transform(X_train)
from sklearn.model_selection import RandomizedSearchCV
# build a classifier
clf = RandomForestClassifier()


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(results['mean_test_score'][candidate], results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {'min_samples_leaf': randint(1,5),
              'min_samples_split': randint(2, 10),
              'n_estimators': randint(100, 300),
              'max_features': randint(2, 13)}

n_iter_search = 15
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=2, iid=False, n_jobs=-1)

In [57]:
random_search.fit(X_train_encoded, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [58]:
report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.785 (std: 0.002)
Parameters: {'max_features': 4, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 102}

Model with rank: 2
Mean validation score: 0.785 (std: 0.000)
Parameters: {'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 252}

Model with rank: 3
Mean validation score: 0.784 (std: 0.001)
Parameters: {'max_features': 6, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 271}



In [61]:
#use this to create submission file

y_pred = pipeline.predict(X_test)
predictions = pd.DataFrame(data=y_pred, index=test['id'])

print(predictions.head())

predictions.columns = ['status_group']
submission = predictions.to_csv('../content/submission.csv')


                    0
id                   
50785      functional
51630      functional
17168      functional
45559  non functional
49871      functional
